In [1]:
import numpy as np
import pandas as pd
import dill
from src.utils import *
from src.augmentator import Augmentator
from src.image_sequence_generator import ImageSequenceGenerator
from src.video_classifier import VideoClassifier
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from matplotlib import pyplot as plt

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
train_ids, train_classes = read_ids('../IDs/train.csv')
valid_ids, valid_classes = read_ids('../IDs/valid.csv')
test_ids, test_classes = read_ids('../IDs/test.csv')

In [3]:
video_path = '../videos_old'
train_augmentator = Augmentator(preprocessing_function=scale_image)
valid_augmentator = Augmentator(preprocessing_function=scale_image)
train_generator = ImageSequenceGenerator(
    video_path,
    train_ids,
    train_classes,
    ['0', '1'],
    train_augmentator,
    fit_eval=True,
    fps=3,
    sequence_time=2,
    batch_size=3
)
valid_generator = ImageSequenceGenerator(
    video_path,
    valid_ids,
    valid_classes,
    ['0', '1'],
    valid_augmentator,
    fit_eval=True,
    fps=3,
    sequence_time=2,
    batch_size=3
)

In [4]:
%%time
video_classifier = VideoClassifier(input_shape=(train_generator.timesteps, 224, 224, 3))
video_classifier.build_model()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Wall time: 24.4 s


In [ ]:
%%time
video_classifier.fit(train_generator, epochs=1000, validation_data=valid_generator)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000


In [ ]:
with open('model/history.pkl', 'wb') as f:
    dill.dump(video_classifier.history, f)

In [ ]:
# %%time
# video_classifier = VideoClassifier(input_shape=(train_generator.timesteps, 224, 224, 3))
# video_classifier.build_model(weights_path='model/checkpoint_best_model.h5')

In [ ]:
# %%time
# with open('model/history.pkl', 'rb') as f:
#     history = dill.load(f)

In [ ]:
plot_learning_curves(history)

In [ ]:
predict_augmentator = Augmentator(
    rotation_range=(0, 0),
    width_shift_range=(0, 0),
    height_shift_range=(0, 0),
    scale_x_range=(1, 1),
    scale_y_range=(1, 1),
    horizontal_flip=False,
    preprocessing_function=scale_image
)

In [ ]:
train_predict_generator = ImageSequenceGenerator(
    video_path,
    train_ids,
    train_classes,
    ['0', '1'],
    predict_augmentator,
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False
)
valid_predict_generator = ImageSequenceGenerator(
    video_path,
    valid_ids,
    valid_classes,
    ['0', '1'],
    predict_augmentator,
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False
)
test_predict_generator = ImageSequenceGenerator(
    video_path,
    test_ids,
    test_classes,
    ['0', '1'],
    predict_augmentator,
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False
)

In [ ]:
%%time
y_true_train, y_pred_train = video_classifier.predict(train_predict_generator)

In [ ]:
%%time
y_true_valid, y_pred_valid = video_classifier.predict(valid_predict_generator)

In [ ]:
%%time
y_true_test, y_pred_test = video_classifier.predict(test_predict_generator)

In [ ]:
fpr_train, tpr_train, roc_thresholds_train = roc_curve(y_true_train[:, 1], y_pred_train[:, 1])
fpr_valid, tpr_valid, roc_thresholds_valid = roc_curve(y_true_valid[:, 1], y_pred_valid[:, 1])
fpr_test, tpr_test, roc_thresholds_test = roc_curve(y_true_test[:, 1], y_pred_test[:, 1])
plot_curves(
    fpr_train,
    tpr_train,
    roc_thresholds_train,
    fpr_valid,
    tpr_valid,
    roc_thresholds_valid,
    fpr_test,
    tpr_test,
    roc_thresholds_test,
    output='plot/ROC_curves.html'
)

In [ ]:
recall_train, precision_train, pr_thresholds_train = precision_recall_curve(y_true_train[:, 1], y_pred_train[:, 1])
recall_valid, precision_valid, pr_thresholds_valid = precision_recall_curve(y_true_valid[:, 1], y_pred_valid[:, 1])
recall_test, precision_test, pr_thresholds_test = precision_recall_curve(y_true_test[:, 1], y_pred_test[:, 1])
plot_curves(
    recall_train,
    precision_train,
    pr_thresholds_train,
    recall_valid,
    precision_valid,
    pr_thresholds_valid,
    recall_test,
    precision_test,
    pr_thresholds_test,
    curve_type='PR',
    output='plot/PR_curves.html'
)

In [ ]:
roc_auc = roc_auc_score(y_true_train[:, 1], y_pred_train[:, 1])
gini = gini_score(roc_auc)
display('ROC AUC and GINI train:', roc_auc, gini)

In [ ]:
roc_auc = roc_auc_score(y_true_valid[:, 1], y_pred_valid[:, 1])
gini = gini_score(roc_auc)
display('ROC AUC and GINI valid:', roc_auc, gini)

In [ ]:
roc_auc = roc_auc_score(y_true_test[:, 1], y_pred_test[:, 1])
gini = gini_score(roc_auc)
display('ROC AUC and GINI test:', roc_auc, gini)

In [ ]:
print('PR Train:', auc(recall_train, precision_train, reorder=True))
print('PR Valid:', auc(recall_valid, precision_valid, reorder=True))
print('PR Test:', auc(recall_test, precision_test, reorder=True))

In [ ]:
mark_video_path = '../videos_old_raw/raw/test/0'
mark_video_generator = ImageSequenceGenerator(
    mark_video_path,
    ids=['Full Match - Arsenal FC vs. FC Bayern 2-1 - International Champions Cup 2019.mp4'],
    fps=3,
    sequence_time=2,
    batch_size=3,
    shuffle=False,
    fit_eval=False
)

In [ ]:
# video_classifier = VideoClassifier(input_shape=(mark_video_generator.timesteps, 224, 224, 3))
# video_classifier.build_model(weights_path='model/checkpoint_best_model.h5')

In [ ]:
%%time
y_pred_mark = video_classifier.predict(mark_video_generator)

In [ ]:
def hms(sec):
    seconds = sec
    hours = int(seconds / 3600)
    seconds = int(seconds - hours * 3600)
    minutes = int(seconds / 60)
    seconds = int(seconds - minutes * 60)
    return hours, minutes, seconds

In [ ]:
trace = go.Scatter(
    x=list(map(lambda x: '{:02}:{:02}:{:02}'.format(x[0], x[1], x[2]), map(hms, range(y_pred_mark.shape[0])))),
    y=y_pred_mark[:, 1]
)
data = [trace]
layout = go.Layout(
    title=dict(
        text='Violation distribution'
    ),
    xaxis=dict(
        title=dict(
            text='Second'
        )
    ),
    yaxis=dict(
        title=dict(
            text='Score per second'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='plot/violation_distribution.html')

In [8]:
# import cv2
# from src.video_stream import VideoStream
# with VideoStream('../videos_old/00001.mp4') as cap:
#     print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
#     print(train_generator.timesteps)
#     print(int(np.ceil(cap.get(cv2.CAP_PROP_FPS)) * train_generator.shift_time))

10993
6
30
